In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson, chisquare
from bettools import generate_seasons, get_data

## This uses the method as outlined at https://www.football-data.co.uk/Contrarian.pdf

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.options.mode.chained_assignment = None

leagues = ["E0", "E1", "E2", "E3", "S0", "SC0", "SC1", "SC2", "SC3"]

seasons = generate_seasons(2012, 2024)

df_ls = get_data(seasons, leagues)

ModuleNotFoundError: No module named 'bettools'

In [ ]:
rated_ls = []

for main_df in df_ls:
    main_df["P_margin"] = (
        1 / main_df["PSH"] + 1 / main_df["PSD"] + 1 / main_df["PSA"]
    ) - 1

    main_df["fair_PSH"] = (3 * main_df["PSH"]) / (
        3 - main_df["P_margin"] * main_df["PSH"]
    )
    main_df["fair_PSD"] = (3 * main_df["PSD"]) / (
        3 - main_df["P_margin"] * main_df["PSD"]
    )
    main_df["fair_PSA"] = (3 * main_df["PSA"]) / (
        3 - main_df["P_margin"] * main_df["PSA"]
    )

    main_df["check_col"] = (
        1 / main_df["fair_PSH"] + 1 / main_df["fair_PSD"] + 1 / main_df["fair_PSA"]
    )

    ratings = {}

    for i in range(len(main_df)):
        home_team = main_df.loc[i]["HomeTeam"]
        away_team = main_df.loc[i]["AwayTeam"]

        if home_team in ratings:
            home_team_prior_rating = ratings[home_team]
        else:
            home_team_prior_rating = 0

        if away_team in ratings:
            away_team_prior_rating = ratings[away_team]
        else:
            away_team_prior_rating = 0

        main_df.loc[i, "home_team_prior_rating"] = home_team_prior_rating
        main_df.loc[i, "away_team_prior_rating"] = away_team_prior_rating

        match_rating = away_team_prior_rating - home_team_prior_rating

        main_df.loc[i, "match_rating"] = match_rating

        if main_df.loc[i]["FTHG"] > main_df.loc[i]["FTAG"]:
            home_team_post_rating = home_team_prior_rating + (
                1 - 1 / main_df.loc[i]["fair_PSH"]
            )
        else:
            home_team_post_rating = home_team_prior_rating + (
                -1 / main_df.loc[i]["fair_PSH"]
            )

        if main_df.loc[i]["FTHG"] < main_df.loc[i]["FTAG"]:
            away_team_post_rating = away_team_prior_rating + (
                1 - 1 / main_df.loc[i]["fair_PSA"]
            )
        else:
            away_team_post_rating = away_team_prior_rating + (
                -1 / main_df.loc[i]["fair_PSA"]
            )
        main_df.loc[i, "away_team_post_rating"] = away_team_post_rating

        ratings[home_team] = home_team_post_rating
        ratings[away_team] = away_team_post_rating

    THRESH = 0

    bet_df = main_df[abs(main_df["match_rating"]) > THRESH]

    home_bet_df = bet_df[bet_df["match_rating"] > 0]
    home_bet_df["prediction"] = "H"
    away_bet_df = bet_df[bet_df["match_rating"] < 0]
    away_bet_df["prediction"] = "A"

    home_bet_df["win"] = np.where(home_bet_df["FTHG"] > home_bet_df["FTAG"], 1, 0)

    # home_bet_df['stake'] = 1/home_bet_df['fair_PSH']
    home_bet_df["stake"] = 1
    home_bet_df["profit"] = (
        home_bet_df["stake"] * home_bet_df["home_max_odds"] * home_bet_df["win"]
    ) - home_bet_df["stake"]

    away_bet_df["win"] = np.where(away_bet_df["FTHG"] < away_bet_df["FTAG"], 1, 0)

    # away_bet_df['stake'] = 1/away_bet_df['fair_PSA']
    away_bet_df["stake"] = 1
    away_bet_df["profit"] = (
        away_bet_df["stake"] * away_bet_df["away_max_odds"] * away_bet_df["win"]
    ) - away_bet_df["stake"]

    result_df = pd.concat([home_bet_df, away_bet_df])

    result_df = result_df.sort_values("Date")

    result_df.reset_index(drop=True, inplace=True)

    rated_ls.append(result_df)

In [ ]:
final_df = pd.concat(rated_ls)

final_df = final_df.sort_values("Date")
final_df.reset_index(drop=True, inplace=True)

final_df

In [ ]:
THRESH = 0

final_df = pd.concat(rated_ls)

final_df = final_df.sort_values("Date")
final_df.reset_index(drop=True, inplace=True)

final_df = final_df[abs(final_df["match_rating"]) > THRESH]

final_df.reset_index(inplace=True, drop=True)

final_df["c_profit"] = final_df["profit"].cumsum()

final_df["div_c_profit"] = final_df.groupby("Div")["profit"].cumsum()

fig, ax = plt.subplots()

for league, data in final_df.groupby("Div"):
    data.plot(x="Date", y="div_c_profit", ax=ax, label=league)

plt.title("Cumulative Profit by Football League")
plt.xlabel("League")
plt.ylabel("Cumulative Profit")
plt.legend()
plt.show()

In [ ]:
THRESH_MIN = 3
THRESH_MAX = 5

final_df = pd.concat(rated_ls)

exclude_list = ["I2", "I1"]
final_df = final_df[~final_df["Div"].isin(exclude_list)]

final_df = final_df[final_df["prediction"] == "H"]

final_df = final_df.sort_values("Date")
final_df.reset_index(drop=True, inplace=True)

final_df = final_df[
    (abs(final_df["match_rating"]) > THRESH_MIN)
    & (abs(final_df["match_rating"]) < THRESH_MAX)
]

final_df.reset_index(inplace=True, drop=True)

final_df["c_profit"] = final_df["profit"].cumsum()

final_df["c_profit"].plot()

In [ ]:
THRESH = 0

bet_df = main_df[abs(main_df["match_rating"]) > THRESH]

home_bet_df = bet_df[bet_df["match_rating"] > 0]
home_bet_df["prediction"] = "H"
away_bet_df = bet_df[bet_df["match_rating"] < 0]
away_bet_df["prediction"] = "A"

home_bet_df["win"] = np.where(home_bet_df["FTHG"] > home_bet_df["FTAG"], 1, 0)

home_bet_df["stake"] = 1 / home_bet_df["fair_PSH"]
home_bet_df["profit"] = (
    home_bet_df["stake"] * home_bet_df["PSH"] * home_bet_df["win"]
) - home_bet_df["stake"]

away_bet_df["win"] = np.where(away_bet_df["FTHG"] < away_bet_df["FTAG"], 1, 0)

away_bet_df["stake"] = 1 / away_bet_df["fair_PSA"]
away_bet_df["profit"] = (
    away_bet_df["stake"] * away_bet_df["PSA"] * away_bet_df["win"]
) - away_bet_df["stake"]

result_df = pd.concat([home_bet_df, away_bet_df])

result_df = result_df.sort_values("Date")

result_df.reset_index(drop=True, inplace=True)

result_df["c_profit"] = result_df["profit"].cumsum()

result_df["c_profit"].plot()